Initialization

Mathematic explanations : https://machinelearnia.com/regression-lineaire-simple/

In [ ]:
# Force external files reloading
%reload_ext autoreload
%autoreload 2
import myvars as v

# Importing modules
import csv
import numpy as np
import matplotlib.pyplot as plt
import os.path
from IPython import display as dp
import time

# Location of the dataset
DATA_LOCATION = '../data/data.csv'

# Creating a fresh theta vars files
theta = v.init(True)

Retrieving Dataset

In [ ]:
# Checking dataset existence
if (os.path.exists(DATA_LOCATION) != True):
	print('Dataset file doesn\'t exists at ' + DATA_LOCATION)
	raise KeyboardInterrupt()

print('We assume that the dataset is correctly formated with valid data')
# Openning data file and parsing it into 2 lists
file = open(DATA_LOCATION, "r")
reader = csv.reader(file)
next(reader)
km = []
price = []
for row in reader:
	km.append(row[0])
	price.append(row[1])

Filling and shaping matixes, then representing dataset

In [ ]:
# Getting matrixes dimmensions infos
m = len(km)
# Creating empty numpy arrays then filling them with values from dataset
x = np.empty((m, 1), float)
i = 0
for num in km:
	x[i, 0] = float(km[i])
	i += 1
y = np.empty((m, 1), float)
i = 0
for num in price:
	y[i, 0] = float(price[i])
	i += 1

print('x is a', x.shape, 'matrix filled with mileages values')
print('y is a', y.shape, 'matrix filled with prices values')

# Printing a graph reprensenting the dataset
print('Data graph :')
plt.scatter(x, y)
plt.title("Data representation", fontsize=20)
plt.xlabel("mileage (km)")
plt.ylabel("price ($)")
plt.show()

Preventing from overflows by normalizing the dataset

In [ ]:
# Normalization to keep 0 <= values <= 1 and preventing future overflows
def normalize(nb):
    return (nb - min(nb)) / (max(nb) - min(nb))

backup = x
x = normalize(x)

Adding bias column and setting random theta values

In [ ]:
# Adding bias column to x
X = np.hstack((x, np.ones(x.shape)))
print('X is a', x.shape, 'matrix filled with mileages values and a bias column')
# Setting random initial theta values
theta = np.random.randn(2, 1)
print('Initial theta values (random), theta0 = ' + str(theta[0]) + ' $, theta1 = ' + str(theta[1]) + ' $/km')

Gradient descent functions definitions

In [ ]:
# TBH, setting the formulae as F = theta0 + theta1 * x is complete garbage. theta0 and theta1 should be inverted
# The model contains all predictions in a single matrix, here is the demonstration
#		F 		=		X 		.	theta	=	theta0 + theta1 * x
#  [predict(0)	   [x(0),	1
#	predict(1)		x(1),	1	   [theta1
#	predict(2)	=	x(2),	1	.	theta0]
#	...				...
#	predict(m)]		x(m),	1]
def model(X, theta):
	return X.dot(theta)

# Difference = (model(X, theta) - y)
# Average difference = 1 / m * sum(differences)
# The cost function returns the mean squared error (average euclidean distance between prediction and value from dataset) divided by 2
def cost_function(X, y, theta):
	m = len(y)
	return 1/(2*m) * np.sum((model(X, theta) - y)**2)

# Note : we are searching the theta values such as MSE with this vector is minimal
# The gradient is the vector representing first derivative of the cost function, meaning that adjusting theta in this direction is minimizing MSE
def grad(X, y, theta):
	m = len(y)
	return 1/m * X.T.dot(model(X, theta) - y)

LEARNING_TRESHOLD = 0.03 # ? Under this ratio, we stop the learning phase

# Proper learning phase, 
def gradient_descent(X, y, theta, learning_rate, n_iterations):
	# Initializing MSE history matrix
	cost_history = []
	# For a (defined in next cell) number of iterations
	for i in range(0, n_iterations):

		# * Uncomment this to show "live" learning
		# plt.scatter(x, y)
		# plt.title("Machine learning", fontsize=20)
		# plt.xlabel("mileage (km, normalized)")
		# plt.ylabel("price ($)")
		# plt.plot(x, model(X, theta), c='r')
		# dp.clear_output(wait=True)
		# dp.display(plt.gcf())
		# time.sleep(0.0005)
		# plt.clf()

		# Correcting theta value by substracting a part of the gradient to move in a way to minimize MSE
		# Learning rate is not high because you don't want to got too far from where you were to prevent from boucing
		# 	from a "side of point with minimal MSE" to the other (look at a basic square function, if you are on the left side of the minimum you want to
		# 	go to the other side but keep approaching iteration by iteration)
		# Learning rate too low would also result in a slow approch, which is not desired either. You need to try some values and see how it goes to adjust
		if (abs(grad(X, y, theta))[0] < abs(theta[0]) / 100 * LEARNING_TRESHOLD) and (abs(grad(X, y, theta))[1] < abs(theta[1]) / 100 * LEARNING_TRESHOLD):
			break # ? Stop if further iterations wouldn't improve the model more than slightly
		theta -= learning_rate * grad(X, y, theta)
		# Adding current MSE to history
		cost_history.append(cost_function(X, y, theta))
	# Returning final adjusted value
	return theta, cost_history

Setting learning variables and representing

In [ ]:
# Number of times the program will iterate
n_iterations = 2000
# Multiplication factor of the MSE to prevent from over reacting
learning_rate = 0.1

# Printing graph and initial function before learning
print('Graph before learning :')
plt.scatter(backup, y)
plt.title("Before learning", fontsize=20)
plt.xlabel("mileage (km)")
plt.ylabel("price ($)")
plt.plot(backup, model(X, theta), c='r')

Learning, saving and showing results, then reprensenting again

In [ ]:
# Learning phase
theta_final, cost_history = gradient_descent(X, y, theta, learning_rate, n_iterations)

In [ ]:
# Filling a matrix with predictions (Predictions = X . theta)
predictions = model(X, theta_final)

# Reverting normalization to get back to initial values
theta_final[0] = theta_final[0] / ((max(backup) - min(backup)) + min(backup))
x = x * (max(backup) - min(backup)) + min(backup)
# Theta file updating
file = open(v.THETA_LOCATION, "w")
file.write(str(theta_final[1]).replace('[', '').replace(']', '') + ',' + str(theta_final[0]).replace('[', '').replace(']', ''))
file.close()
# Final theta values report
print('Final theta values, theta0 = ' + str(theta_final[1]) + ' $, theta1 = ' + str(theta_final[0]) + ' $/km')

# Printing graph and function after learning
print('Graph after learning :')
plt.scatter(x, y)
plt.title("After learning", fontsize=20)
plt.xlabel("mileage (km)")
plt.ylabel("price ($)")
plt.plot(x, predictions, c='r')
plt.show()

Representing cost function history (mean squared error evolution over iterations)

In [ ]:
# Representing learning curve / MSE over iterations
print('Learning curve :')
j = -1
PLOT_RATIO = 15 # ? Cap ratio between min and max cost_history values
for i in range (0, len(cost_history)):
    if cost_history[i] > PLOT_RATIO * cost_history[-1]:
        cost_history[i] = PLOT_RATIO * cost_history[-1]
        j = i
if j > -1:
	print('First', j, 'values manually capped to', PLOT_RATIO * cost_history[-1])
print('The learning has processed to', len(cost_history), 'iterations')
plt.plot(range(len(cost_history)), cost_history)
plt.title("Mean squared error evolution over iterations\n", fontsize=20)
plt.xlabel("iterations")
plt.ylabel("MSE")
plt.show()

In [ ]:
# Calculating determination coefficient and printing it
u = ((y - predictions) ** 2).sum()
v = ((y - y.mean()) ** 2).sum()
print('Determination cofficient :', 1 - u / v)